In [1]:
#Main Focus

# The super() function
#The pitfalls of subclassing from builtin types
#Multiple inheritance and method resolution order
#Mixin classes

The Super() Function

In [3]:
class DoppelDict(dict):
    def __setitem__(self, __key, __value) -> None:
        return super().__setitem__(__key, [__value]*2)

In [9]:
dd = DoppelDict(one=1)
dd #doesnt trigger the modified setitem
dd['two'] = 2
dd    #Triggers the setitem method
dd.update(three=3),dd

#calls the inbuilt dict methods

(None, {'one': 1, 'two': [2, 2], 'three': 3})

In [10]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [15]:
#consider this
ad = AnswerDict(one=1)
ad['one']  #triggers the subclass method

d = {}
d.update(ad)
d['one']   #Triggers the builtin method
d

{'one': 1}

Instead of using the built in classes as superclass use the UserList, UserDict and UserString which are disigned to be easily extended.

In [16]:
#if you subclass collections.UserDict instead of dict the issues wxposed in the previous examples which inherits the builtin class is solved

In [17]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self,key,value):
        return super().__setitem__(key, [value]*2)

In [23]:
dd = DoppelDict2(one=1)
dd['two'] = 2
dd
dd.update(three=3)
dd  #overrides the default inbuilt method

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [24]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return super().__dict__

In [27]:
ad = AnswerDict2(one=1)
ad['one'] #show super().__dict__

{'data': {'one': 1}}

Multiple Inheritance and Method Resolutiion Order

In [29]:
"""Any language implementing multiple inheritance needs to deal with potential naming conflicts when superclasses implement a method by the same name. This is called the diamond problem"""

'Any language implementing multiple inheritance needs to deal with potential naming conflicts when superclasses implement a method by the same name. This is called the diamond problem'

In [30]:
class Root:  
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'

In [31]:
r = Root()
r.pong()

In [35]:
class A(Root):  
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

In [39]:
r = A()
r.pong()

<instance of A>.pong() in A
<instance of A>.pong() in Root


In [41]:
class B(Root):  
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')

In [42]:
class Leaf(A, B):  
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [46]:
s = B()
t = Leaf()
t.pong

<bound method A.pong of <instance of Leaf>>

In [47]:
leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [48]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [50]:
#Every class has an attribute called __mro__ holding a tuple of references to the superclasses in method resolution oreder, form the current class all the way to the object class. For the leaf class thsi si the __mro__

Leaf.__mro__   #Show inherited superclasses

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

In [69]:
#Surprising result of the dynamic behavior of python

class U():
    def ping(self):
        #print(f'{self}.ping() in U')
        super().ping()

In [70]:
u = U()
U.__mro__

(__main__.U, object)

In [71]:
class LeafUA(U,A):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

In [72]:
lua = LeafUA()
lua.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [73]:
import tkinter

In [2]:
def print_mro(cls):
    print(','.join(c.__name__ for c in cls.__mro__))

In [76]:
print_mro(tkinter.Text)

Text,Widget,BaseWidget,Misc,Pack,Place,Grid,XView,YView,object


MIXIN CLASSES

In [77]:
import collections

def _upper(key):  
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

In [81]:
ucm = UpperCaseMixin()

In [82]:
ucm['one'] = 1

AttributeError: 'super' object has no attribute '__setitem__'

In [92]:
#Calling the class individually wont do anything but mixins must appear first in the tuple o fbase classes in a calss declaration


class UpperDict(UpperCaseMixin, collections.UserDict):
    pass
ud = UpperDict()
ud['one'] = 1
ud.get('one')

1

In [89]:
class UpperCounter(UpperCaseMixin, collections.Counter):
    """Specialized counter that uppercases string keys"""

In [91]:
uc = UpperCounter()
uc.__doc__

'Specialized counter that uppercases string keys'

In [103]:
#quick demonstration of uppercounter

c = UpperCounter('BaNanA')
c.__class__.__bases__

(__main__.UpperCaseMixin, collections.Counter)

In [3]:
import tkinter

print_mro(tkinter.Toplevel)

Toplevel,BaseWidget,Misc,Wm,object


In [4]:
print_mro(tkinter.Widget)

Widget,BaseWidget,Misc,Pack,Place,Grid,object


In [5]:
print_mro(tkinter.Button)

Button,Widget,BaseWidget,Misc,Pack,Place,Grid,object


In [6]:
print_mro(tkinter.Entry)

Entry,Widget,BaseWidget,Misc,Pack,Place,Grid,XView,object


In [7]:
print_mro(tkinter.Text)

Text,Widget,BaseWidget,Misc,Pack,Place,Grid,XView,YView,object


Coping with Inheritance

In [8]:
#Favor Object Composition over Class Inheritance

Multiple Inheritance in the Real World

![Alt text](Screenshot%20from%202022-12-03%2009-42-06.png)

Avoid Subclassing form Concrete Classes